## Part 4- Pytorch Lightning Implementation 

Created by Michael Casey, Andrew Cruez, Peter Stewart, and Hemraj Yadav

__About Part 4__

A continuation of the HCDR classification project, this part will focus on implimentation of an artificail neural network using Pytorch. We will use the same datasets created in part 2 but in this case we'll model exclusively using PyTorch Lightning.

In [1]:
import lightning 
import pytorch_lightning as pl
import torch 
import torch.nn as nn 
from lightning import Callback
from pytorch_lightning.callbacks import ModelCheckpoint
from torchmetrics import Accuracy
import sys
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import pandas as pd
import pickle
# from torchvision import transforms
sys.path.insert(0, '..')



/home/pete/Documents/ML_code/mlenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Load data from part 3
DATA_DIR = "./Data"

with open(f"{DATA_DIR}/X_train.pkl", 'rb') as aa:
    X_train = pickle.load(aa)

with open(f"{DATA_DIR}/X_test.pkl", 'rb') as bb:
    X_test = pickle.load(bb)

with open(f"{DATA_DIR}/y_train.pkl", 'rb') as cc:
    y_train = pickle.load(cc)

with open(f"{DATA_DIR}/y_test.pkl", 'rb') as dd:
    y_test = pickle.load(dd)

with open(f"{DATA_DIR}/X_valid.pkl", 'rb') as ee:
    X_valid = pickle.load(ee)

with open(f"{DATA_DIR}/y_valid.pkl", 'rb') as ff:
    y_valid = pickle.load(ff)

with open(f"{DATA_DIR}/X_test_SA.pkl", 'rb') as gg:
    X_test_SA = pickle.load(gg)

# Pytorch Preprocessing Pipeline

In [3]:
#Now let's determine which features are categorical or numerical for our X_train_FT dataset
numerical_at = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_at = X_train.select_dtypes(include=['object', 'bool']).columns
X_train[categorical_at]=X_train[categorical_at].astype(str)
X_test[categorical_at]=X_test[categorical_at].astype(str)
X_valid[categorical_at]=X_valid[categorical_at].astype(str)
tot_from_above = (len(numerical_at)+len(categorical_at))
print (f"X_train col number is: {X_train.shape[1]}")
print (f"X_test col number is: {X_test.shape[1]}")
print (f"X_valid col number is: {X_valid.shape[1]}")
print (f"Total of num and cat columns is: {tot_from_above}")
X_train.info()

X_train col number is: 1297
X_test col number is: 1297
X_valid col number is: 1297
Total of num and cat columns is: 1297
<class 'pandas.core.frame.DataFrame'>
Int64Index: 209107 entries, 35339 to 212146
Columns: 1297 entries, NAME_CONTRACT_TYPE to SUM(previous_FT.SELLERPLACE_AREA)
dtypes: float64(1233), int64(44), object(20)
memory usage: 2.0+ GB


In [4]:
BL_num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
    ])

BL_cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ohe', OneHotEncoder(handle_unknown="ignore"))
    ])

# MNB_num_pipeline = Pipeline([
#         ('imputer', SimpleImputer(strategy='constant',fill_value= 0)),
#         ('scaler', MinMaxScaler()),
#     ])

# new_num_pipeline = Pipeline([
#         ('imputer', SimpleImputer(strategy='constant',fill_value= 0)),
#         ('scaler', MinMaxScaler()),
#     ])

In [5]:

torch_pipeline = ColumnTransformer(transformers=[
       ("num_pipeline", BL_num_pipeline, numerical_at),
       ("cat_pipeline", BL_cat_pipeline, categorical_at)],
       remainder='drop',
        n_jobs=1
   )
X_train_transformed=  torch_pipeline.fit_transform(X_train)


column_names = list(numerical_at) + list(torch_pipeline.transformers_[1][1].named_steps["ohe"].get_feature_names_out(categorical_at))
X_train_transformed= (pd.DataFrame(X_train_transformed,  columns=column_names))
number_of_inputs = X_train_transformed.shape[1]

In [6]:
X_test_transformed= torch_pipeline.fit_transform(X_test)


column_names = list(numerical_at) + list(torch_pipeline.transformers_[1][1].named_steps["ohe"].get_feature_names_out(categorical_at))
X_test_transformed= (pd.DataFrame(X_test_transformed,  columns=column_names))
number_of_inputs = X_test_transformed.shape[1]

In [7]:
X_valid_transformed= torch_pipeline.fit_transform(X_valid)


column_names = list(numerical_at) + list(torch_pipeline.transformers_[1][1].named_steps["ohe"].get_feature_names_out(categorical_at))
X_valid_transformed= (pd.DataFrame(X_valid_transformed,  columns=column_names))
number_of_inputs = X_valid_transformed.shape[1]

In [8]:
drop_list = (X_train_transformed.columns.difference(X_test_transformed.columns))
print(len(drop_list))
drop_list
X_train_transformed = X_train_transformed.drop(drop_list, axis=1)
X_train_transformed.shape

0


(209107, 1431)

In [9]:
drop_list = (X_train_transformed.columns.difference(X_valid_transformed.columns))
print(len(drop_list))
X_train_transformed = X_train_transformed.drop(drop_list, axis=1)
X_train_transformed.shape

2


(209107, 1429)

In [10]:
drop_list = (X_test_transformed.columns.difference(X_valid_transformed.columns))
print(len(drop_list))
X_test_transformed = X_test_transformed.drop(drop_list, axis=1)
X_test_transformed.shape

2


(46127, 1429)

In [11]:
print(X_train_transformed.shape)
print(X_test_transformed.shape)
print(X_valid_transformed.shape)

(209107, 1429)
(46127, 1429)
(52277, 1429)


In [ ]:
X_train_transformed.to_csv(f'{DATA_DIR}/X_train_pt.csv',index=False)
X_test_transformed.to_csv(f'{DATA_DIR}/X_test_pt.csv',index=False)
X_valid_transformed.to_csv(f'{DATA_DIR}/X_valid_pt.csv',index=False)

In [25]:
y_train.to_csv(f'{DATA_DIR}/y_train_pt.csv',index=False)
y_test.to_csv(f'{DATA_DIR}/y_test_pt.csv',index=False)
y_valid.to_csv(f'{DATA_DIR}/y_valid_pt.csv',index=False)

In [12]:
print(X_train_transformed.shape)

(209107, 1429)


In [13]:
class MultiLayerPerceptron(pl.LightningModule):
    def __init__(self, input_size=(1429), hidden_units=(32, 16)):
        super().__init__()
        
        # new PL attributes:
        self.train_acc = Accuracy("binary", num_classes=2)  #change to binary class and 2 for HCDR
        self.valid_acc = Accuracy("binary", num_classes=2)
        self.test_acc = Accuracy("binary",num_classes=2)
        
        # Model similar to previous section:parse=False
        input_size = input_size
        all_layers = [nn.Linear(input_size, hidden_units[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_units[0], hidden_units[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_units[1], 2)
                      ]
        # for hidden_unit in hidden_units: 
 
        # all_layers.append(nn.Linear(hidden_units[-1], 10)) 
        self.model = nn.Sequential(*all_layers)

    def forward(self, x):
        x = self.model(x)
        return x

    def training_step(self, batch, batch_idx):
        # add in a loss layer
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y.squeeze().long())
        preds = torch.argmax(logits, dim=1)
        self.train_acc.update(preds, y.squeeze().long())
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def on_train_epoch_end(self):
        self.log("train_acc", self.train_acc.compute())
        self.train_acc.reset()
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y.squeeze().long())
        preds = torch.argmax(logits, dim=1)
        self.valid_acc.update(preds, y.squeeze().long())
        self.log("valid_loss", loss, prog_bar=True)
        return loss
    
    def on_validation_epoch_end(self):
        self.log("valid_acc", self.valid_acc.compute(), prog_bar=True)
        self.valid_acc.reset()

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y.squeeze().long())
        preds = torch.argmax(logits, dim=1)
        self.test_acc.update(preds, y.squeeze().long())
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", self.test_acc.compute(), prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

In [14]:
import pandas as pd

class HCDR_DataModule(pl.LightningDataModule):
    """ Data module for HCDR
    Load data from CSV and convert into a numpy array, 
    then into Pytorch Tensor 
    and then into a Tensor Dataset
    """
    def __init__(self, data_path=f"{DATA_DIR}/", batchsize = 2**14):
        super().__init__()
        self.data_path = data_path
        self.batchsize = batchsize
        #self.transform = transforms.Compose([transforms.ToTensor()])
        
    def prepare_data(self):
        pass 

    def setup(self, stage=None):


        df = pd.read_csv(self.data_path + "X_train_pt.csv")
        df.reset_index()
        df.fillna(0)
        df = df.astype(np.float32)
        X = torch.Tensor(df.values)
        df = pd.read_csv(self.data_path + "y_train_pt.csv")
        df.reset_index()
        np.nan_to_num(df, nan=0)
        df = df.astype(np.float32)
        y = torch.Tensor(df.values.reshape(-1,1))
        self.train = torch.utils.data.TensorDataset(X, y)
        df = pd.read_csv(self.data_path + "X_valid_pt.csv")
        df.reset_index()
        df.fillna(0)
        df = df.astype(np.float32)
        X = torch.Tensor(df.values)
        df = pd.read_csv(self.data_path + "y_valid_pt.csv")
        df.reset_index()
        np.nan_to_num(df, nan=0)
        df = df.astype(np.float32)      
        y = torch.Tensor(df.values.reshape(-1,1))
        self.val = torch.utils.data.TensorDataset(X, y)
        df = pd.read_csv(self.data_path + "X_test_pt.csv")
        df.reset_index()
        df.fillna(0)
        df = df.astype(np.float32)      
        X = torch.Tensor(df.values)
        df = pd.read_csv(self.data_path + "y_test_pt.csv")
        df.reset_index()
        np.nan_to_num(df, nan=0)
        df = df.astype(np.float32)
        y = torch.Tensor(df.values.reshape(-1,1))
        self.test = torch.utils.data.TensorDataset(X, y)
 


    def train_dataloader(self):
        return DataLoader(self.train, batch_size=self.batchsize, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.val, batch_size=self.batchsize, num_workers=4)

    def test_dataloader(self):
        return DataLoader(self.test, batch_size=self.batchsize, num_workers=4)
    
    
torch.manual_seed(1) 

In [15]:
if torch.cuda.is_available(): # if you have GPUs
    print("GPU is available")
    print(torch.__version__)
else:
    print("no gpu")
    print(torch.__version__)

no gpu
2.4.0+cu121


In [21]:
hcdrclassifier = MultiLayerPerceptron()

callbacks = [ModelCheckpoint(save_top_k=1, mode='max', monitor="valid_acc")] # save top 1 model

if torch.cuda.is_available(): # if you have GPUs
    print("GPU is available")
    trainer = pl.Trainer(max_epochs=20, callbacks=callbacks)
else:
    trainer = pl.Trainer(max_epochs=20, callbacks=callbacks)

trainer.fit(model=hcdrclassifier, datamodule=HCDR_DataModule())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type           | Params | Mode 
-----------------------------------------------------
0 | train_acc | BinaryAccuracy | 0      | train
1 | valid_acc | BinaryAccuracy | 0      | train
2 | test_acc  | BinaryAccuracy | 0      | train
3 | model     | Sequential     | 46.3 K | train
-----------------------------------------------------
46.3 K    Trainable params
0         Non-trainable params
46.3 K    Total params
0.185     Total estimated model params size (MB)
9         Modules in train mode
0         Modules in eval mode


/home/pete/Documents/ML_code/mlenv/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (13) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 49: 100%|██████████| 13/13 [00:02<00:00,  5.19it/s, v_num=10, train_loss=0.221, valid_loss=0.252, valid_acc=0.916]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 13/13 [00:02<00:00,  5.19it/s, v_num=10, train_loss=0.221, valid_loss=0.252, valid_acc=0.916]


In [22]:
trainer.test(model=hcdrclassifier, datamodule=HCDR_DataModule(), ckpt_path='best')

Restoring states from the checkpoint path at /home/pete/Documents/ML_code/ML_Classification_HCDR/lightning_logs/version_10/checkpoints/epoch=9-step=130.ckpt
Loaded model weights from the checkpoint at /home/pete/Documents/ML_code/ML_Classification_HCDR/lightning_logs/version_10/checkpoints/epoch=9-step=130.ckpt


Testing DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 33.54it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │      0.9185791015625      │
│         test_loss         │    0.24634385108947754    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.24634385108947754, 'test_acc': 0.9185791015625}]

In [24]:

from torchmetrics import AUROC
class MultiLayerPerceptron3(pl.LightningModule):
    def __init__(self, input_size=(1429), hidden_units=(32, 16)):
        super().__init__()
        
        # new PL attributes:
        # self.train_acc = Accuracy("binary", num_classes=2)  #change to binary class and 2 for HCDR
        self.train_auc = AUROC("binary", num_classes=2)
        # self.test_acc = Accuracy("binary",num_classes=2)
        # self.test_auc = AUROC("binary", num_classes=2)
        self.valid_auc = AUROC("binary", num_classes=2)
        
        # Model similar to previous section:
        input_size = input_size
        all_layers = [nn.Linear(input_size, hidden_units[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_units[0], hidden_units[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_units[1], 2)
                      ]
        # for hidden_unit in hidden_units: 
 
        # all_layers.append(nn.Linear(hidden_units[-1], 10)) 
        self.model = nn.Sequential(*all_layers)

    def forward(self, x):
        x = self.model(x)
        return x

    def training_step(self, batch, batch_idx):
        # add in a loss layer
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y.squeeze().long())
        preds = torch.argmax(logits, dim=1)
        # self.train_acc.update(preds, y.squeeze().long())
        self.train_auc.update(preds, y.squeeze().long())
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def on_train_epoch_end(self):
        # self.log("train_acc", self.train_acc.compute())
        self.log("train_auc", self.train_auc.compute())
        # self.train_acc.reset()
        self.train_auc.reset()
        
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y.squeeze().long())
        preds = torch.argmax(logits, dim=1)
        self.valid_auc.update(preds, y.squeeze().long())
        self.log("valid_loss", loss, prog_bar=True)
        return loss
    
    def on_validation_epoch_end(self):
        self.log("valid_auc", self.valid_auc.compute(), prog_bar=True)
        self.valid_auc.reset()

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y.squeeze().long())
        preds = torch.argmax(logits, dim=1)
        # self.test_acc.update(preds, y.squeeze().long())
        # self.test_auc.update(preds, y.squeeze().long())
        self.log("test_loss", loss, prog_bar=True)
        # self.log("test_acc", self.test_acc.compute(), prog_bar=True)
        # self.log("test_auc", self.test_auc.compute())
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

In [25]:
hcdrclassifier = MultiLayerPerceptron3()

callbacks = [ModelCheckpoint(save_top_k=1, mode='max', monitor="valid_auc")] # save top 1 model

if torch.cuda.is_available(): # if you have GPUs
    print("GPU is available")
    trainer = pl.Trainer(max_epochs=20, callbacks=callbacks, gpus=1)
else:
    trainer = pl.Trainer(max_epochs=20, callbacks=callbacks)

trainer.fit(model=hcdrclassifier, datamodule=HCDR_DataModule())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



  | Name      | Type        | Params | Mode 
--------------------------------------------------
0 | train_auc | BinaryAUROC | 0      | train
1 | valid_auc | BinaryAUROC | 0      | train
2 | model     | Sequential  | 46.3 K | train
--------------------------------------------------
46.3 K    Trainable params
0         Non-trainable params
46.3 K    Total params
0.185     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 13/13 [08:23<00:00,  0.03it/s, v_num=18, train_loss=0.479, valid_loss=0.461, valid_auc=0.500]


/home/pete/Documents/ML_code/mlenv/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (13) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 19: 100%|██████████| 13/13 [00:02<00:00,  4.44it/s, v_num=20, train_loss=0.235, valid_loss=0.250, valid_auc=0.510]

`Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 13/13 [00:02<00:00,  4.41it/s, v_num=20, train_loss=0.235, valid_loss=0.250, valid_auc=0.510]


In [26]:
trainer.test(model=hcdrclassifier, datamodule=HCDR_DataModule(), ckpt_path='best')

Restoring states from the checkpoint path at /home/pete/Documents/ML_code/ML_Classification_HCDR/lightning_logs/version_20/checkpoints/epoch=19-step=260.ckpt
Loaded model weights from the checkpoint at /home/pete/Documents/ML_code/ML_Classification_HCDR/lightning_logs/version_20/checkpoints/epoch=19-step=260.ckpt


Testing DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 32.66it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.2442777454853058     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.2442777454853058}]

In [ ]:
# Start tensorboard
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

In [21]:
from torchmetrics import AUROC

class MultiLayerPerceptron_2(pl.LightningModule):
    def __init__(self, input_size=(1429), hidden_units=(40, 24, 16)):
        super().__init__()
        
        # new PL attributes:
        self.train_acc = Accuracy("binary", num_classes=2)  #change to binary class and 2 for HCDR
        self.valid_acc = Accuracy("binary", num_classes=2)
        self.test_acc = Accuracy("binary",num_classes=2)
        
        # Model similar to previous section:
        input_size = input_size
        all_layers = [nn.Linear(input_size, hidden_units[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_units[0], hidden_units[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_units[1], hidden_units[2]),
                      nn.ReLU(),
                      nn.Linear(hidden_units[2], 2),
                      nn.Sigmoid()
                      ]
        # for hidden_unit in hidden_units: 
 
        # all_layers.append(nn.Linear(hidden_units[-1], 10)) 
        self.model = nn.Sequential(*all_layers)

    def forward(self, x):
        x = self.model(x)
        return x

    def training_step(self, batch, batch_idx):
        # add in a loss layer
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y.squeeze().long())
        preds = torch.argmax(logits, dim=1)
        self.train_acc.update(preds, y.squeeze().long())
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def on_train_epoch_end(self):
        self.log("train_acc", self.train_acc.compute())
        self.train_acc.reset()
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y.squeeze().long())
        preds = torch.argmax(logits, dim=1)
        self.valid_acc.update(preds, y.squeeze().long())
        self.log("valid_loss", loss, prog_bar=True)
        return loss
    
    def on_validation_epoch_end(self):
        self.log("valid_acc", self.valid_acc.compute(), prog_bar=True)
        self.valid_acc.reset()

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y.squeeze().long())
        preds = torch.argmax(logits, dim=1)
        self.test_acc.update(preds, y.squeeze().long())
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", self.test_acc.compute(), prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        return optimizer

In [22]:

hcdrclassifier_2 = MultiLayerPerceptron_2()

callbacks = [ModelCheckpoint(save_top_k=1, mode='max', monitor="valid_acc")] # save top 1 model

if torch.cuda.is_available(): # if you have GPUs
    print("GPU is available")
    trainer = pl.Trainer(max_epochs=15, callbacks=callbacks, gpus=1)
else:
    trainer = pl.Trainer(max_epochs=25, callbacks=callbacks)

trainer.fit(model=hcdrclassifier_2, datamodule=HCDR_DataModule())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type           | Params | Mode 
-----------------------------------------------------
0 | train_acc | BinaryAccuracy | 0      | train
1 | valid_acc | BinaryAccuracy | 0      | train
2 | test_acc  | BinaryAccuracy | 0      | train
3 | model     | Sequential     | 58.6 K | train
-----------------------------------------------------
58.6 K    Trainable params
0         Non-trainable params
58.6 K    Total params
0.234     Total estimated model params size (MB)
12        Modules in train mode
0         Modules in eval mode


/home/pete/Documents/ML_code/mlenv/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (13) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 24: 100%|██████████| 13/13 [00:02<00:00,  5.39it/s, v_num=22, train_loss=0.392, valid_loss=0.397, valid_acc=0.916]

`Trainer.fit` stopped: `max_epochs=25` reached.


Epoch 24: 100%|██████████| 13/13 [00:02<00:00,  5.39it/s, v_num=22, train_loss=0.392, valid_loss=0.397, valid_acc=0.916]


In [23]:
trainer.test(model=hcdrclassifier_2, datamodule=HCDR_DataModule(), ckpt_path='best')

Restoring states from the checkpoint path at /home/pete/Documents/ML_code/ML_Classification_HCDR/lightning_logs/version_22/checkpoints/epoch=1-step=26.ckpt
Loaded model weights from the checkpoint at /home/pete/Documents/ML_code/ML_Classification_HCDR/lightning_logs/version_22/checkpoints/epoch=1-step=26.ckpt


Testing DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 29.68it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9186270236968994     │
│         test_loss         │    0.5212451219558716     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5212451219558716, 'test_acc': 0.9186270236968994}]